<a href="https://colab.research.google.com/github/MomoSho/aiml_tutorials_sklearn/blob/master/TextAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# LSTM and CNN for sequence classification in the IMDB dataset
import numpy
import os
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Bidirectional


#from sklearn.p
import sklearn.metrics
import matplotlib.pyplot as plt

FONTSIZE = 18
FIGURE_SIZE = (10,4)
FIGURE_SIZE2 = (10,10)

In [12]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import fnmatch
import matplotlib.pyplot as plt

import keras
from keras.layers import LeakyReLU, UpSampling1D, Input, Reshape, Activation, Lambda, AveragePooling1D
from keras.layers import Convolution2D, Dense, MaxPooling2D, Flatten, BatchNormalization, Dropout, Conv2DTranspose
from keras.models import Sequential
from keras.optimizers import RMSprop
import random
from keras import Sequential
from keras.preprocessing.sequence import pad_sequences

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.preprocessing.text import one_hot

import re

In [13]:
top_words = 1000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [14]:
X_train.shape

(25000,)

In [15]:
y_train.shape

(25000,)

In [16]:
trainX = np.concatenate((X_train, X_test))
trainY = np.concatenate((y_train, y_test))

In [28]:
Result = np.empty((0,5), float)

In [29]:
def compute_accuracy(model, ins, outs):
    scores = model.evaluate(ins, outs, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    return scores[1]*100


def Retrieve_Data(Data_Fold, data_input, data_output):
    Ins_Data = []
    Outs_Data = []
    
    for Fold_No in  Data_Fold:
        print("Fold No = ", Fold_No)
        Range_Initial = Fold_No * 5000
        Range_final = (Fold_No + 1) * 5000

        for i in range (Range_Initial, Range_final):
            Ins_Data.append(data_input[i])
            Outs_Data.append(data_output[i])
    
    return Ins_Data, Outs_Data

In [30]:
# Linear activation used
def Model_Test(ins_training, outs_training, ins_val, outs_val, ins_test, outs_test, lambd, epochs, patience_epochs, vocab_size, Max_Words_Review):
    
    Ins_Train_encode = sequence.pad_sequences(ins_training, maxlen=Max_Words_Review)
    Ins_Test_encode = sequence.pad_sequences(ins_test, maxlen=Max_Words_Review)
    Ins_Val_encode = sequence.pad_sequences(ins_val, maxlen=Max_Words_Review)
    
    embedding_vector_length = 32 
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_vector_length, input_length = Max_Words_Review))
    
    if (lambd == 1):
        model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Dropout(0.2))
        model.add(LSTM(150))
        model.add(Dropout(0.2))
        
    if (lambd == 2):
        model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='elu'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(LSTM(150, dropout = 0.2))

    if (lambd == 3):
        model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='elu'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(LSTM(150, recurrent_dropout = 0.2))
        
    if (lambd == 4):
        model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='elu'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Bidirectional(LSTM(150, recurrent_dropout = 0.2)))
        
    model.add(Dense(1, activation='sigmoid'))
    
    #print(model.summary())
    
    opt = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='binary_crossentropy', optimizer= opt, metrics=['accuracy'])
    
    
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', 
       min_delta=0.0001, patience= patience_epochs, 
       verbose=0, mode='auto', 
       baseline=None, 
       restore_best_weights=True)

    model.fit(x=np.array(Ins_Train_encode), y=np.array(outs_training),
          validation_data = (np.array(Ins_Val_encode), np.array(outs_val)),
          epochs=epochs, callbacks=[early_stopping], batch_size=64)
    
    predict_Training = model.predict(Ins_Train_encode)
    predict_Validation = model.predict(Ins_Val_encode)
    predict_Test = model.predict(Ins_Test_encode)
    
    train_accuracy = compute_accuracy(model, Ins_Train_encode, outs_training)
    validation_accuracy = compute_accuracy(model, Ins_Val_encode, outs_val)
    test_accuracy = compute_accuracy(model, Ins_Test_encode, outs_test)
    
    return train_accuracy, validation_accuracy, test_accuracy

In [31]:
def all_rotations(data_input, data_output, lambd, No_training_folds, Total_folds, i_th_rotation, epochs, patience_epochs, vocab_size, Max_Words_Review):
    
    global Result
    
    rotation = 0
    Count_of_rotations = 0
    
    Accuracy_Train_Mean = 0
    Accuracy_Val_Mean = 0
    Accuracy_Test_Mean = 0
    
    while rotation < Total_folds:
        
        folds_training = (np.array(range(No_training_folds)) + rotation) % Total_folds
        print("folds training =",folds_training)

        folds_val = []
        folds_test = []

        validation_fold = ((Total_folds - 2) + rotation) % Total_folds
        folds_val.append(validation_fold)
        print("folds Validation =",folds_val)
       
        test_fold = ((Total_folds - 1) + rotation) % Total_folds
        folds_test.append(test_fold)
        print("folds Test =",folds_test)

#         ins_training = np.concatenate(np.take(MI, folds_training))
#         outs_training = np.concatenate(np.take(torque, folds_training))[:,1].reshape(-1,1)

        Ins_Train, Outs_Train = Retrieve_Data(folds_training, data_input, data_output) 
        Ins_Val, Outs_Val = Retrieve_Data(folds_val, data_input, data_output) 
        Ins_Test, Outs_Test = Retrieve_Data(folds_test, data_input, data_output)
        
        Ins_Train = np.array(Ins_Train)
        Outs_Train = np.array(Outs_Train)
        Ins_Val = np.array(Ins_Val)
        Outs_Val = np.array(Outs_Val)
        Ins_Test = np.array(Ins_Test)
        Outs_Test = np.array(Outs_Test)
        #ins_val = np.concatenate(np.take(MI, folds_val))
        #outs_val = np.concatenate(np.take(torque, folds_val))[:,1].reshape(-1,1)

        #ins_test = np.concatenate(np.take(MI, folds_test))
        #outs_test = np.concatenate(np.take(torque, folds_test))[:,1].reshape(-1,1)

        a,b,c = Model_Test(Ins_Train, Outs_Train, Ins_Val, Outs_Val, Ins_Test, Outs_Test, lambd, epochs, patience_epochs, vocab_size, Max_Words_Review)
        print("train_accuracy=",a, "validation_accuracy=",b, "test_accuracy=",c)
        
        Accuracy_Train_Mean = Accuracy_Train_Mean + a
        Accuracy_Val_Mean = Accuracy_Val_Mean + b
        Accuracy_Test_Mean = Accuracy_Test_Mean + c
        
        Count_of_rotations = Count_of_rotations + 1
        print("Rotation before: ")
        print(rotation)
        rotation = rotation + i_th_rotation
        print("Rotation after: ")
        print(rotation)
        
    Accuracy_Train_Mean = Accuracy_Train_Mean / Count_of_rotations
    Accuracy_Val_Mean = Accuracy_Val_Mean / Count_of_rotations
    Accuracy_Test_Mean = Accuracy_Test_Mean / Count_of_rotations
    
    Result = np.append(Result, np.array([[No_training_folds,lambd, Accuracy_Train_Mean, Accuracy_Val_Mean, Accuracy_Test_Mean]]), axis=0)
    return Accuracy_Train_Mean, Accuracy_Val_Mean, Accuracy_Test_Mean

In [32]:
Regularization_Values = [1, 2, 3, 4]
Training_Folds = [1, 2, 3, 4, 5, 6, 7, 8]

In [33]:
# Our cross validation includes the set of four values of lambda where we are testing with different models
def Cross_Validation(Regularization_Values, Training_Folds, Total_folds, i_th_rotation, epochs, patience_epochs, vocab_size, Max_Words_Review):
    (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)
    data_input = numpy.concatenate((X_train, X_test))
    data_output = numpy.concatenate((y_train, y_test))
    for k_fold in Training_Folds[:]:
        for reg_para in Regularization_Values[:]:
            all_rotations(data_input, data_output, reg_para, k_fold, Total_folds, i_th_rotation, epochs, patience_epochs, vocab_size, Max_Words_Review)
        print("Completed Training ", Training_Folds,"  ***************************************************************************")
    return(Result)

In [ ]:
Performance = Cross_Validation(Regularization_Values, Training_Folds, 10, 2, 20, 10, 5000, 150)

folds training = [0]
folds Validation = [8]
folds Test = [9]
Fold No =  0
Fold No =  8
Fold No =  9
Epoch 1/20
79/79 [==============================] - 17s 209ms/step - loss: 0.6933 - accuracy: 0.4982 - val_loss: 0.6929 - val_accuracy: 0.4980
Epoch 2/20
79/79 [==============================] - 16s 203ms/step - loss: 0.6924 - accuracy: 0.5106 - val_loss: 0.6926 - val_accuracy: 0.4948
Epoch 3/20
79/79 [==============================] - 17s 211ms/step - loss: 0.6907 - accuracy: 0.5496 - val_loss: 0.6888 - val_accuracy: 0.5150
Epoch 4/20
79/79 [==============================] - 16s 204ms/step - loss: 0.6407 - accuracy: 0.6394 - val_loss: 0.6078 - val_accuracy: 0.6574
Epoch 5/20
79/79 [==============================] - 16s 205ms/step - loss: 0.5100 - accuracy: 0.7548 - val_loss: 0.5187 - val_accuracy: 0.7516
Epoch 6/20
79/79 [==============================] - 16s 203ms/step - loss: 0.4160 - accuracy: 0.8186 - val_loss: 0.5073 - val_accuracy: 0.7666
Epoch 7/20
79/79 [========================

In [ ]:
Result_df = pd.DataFrame(Result)

In [ ]:
Result_df.rename(columns={0: 'Number of Folds', 1: 'Regularization Parameter Value', 
                          2: 'Mean_Train_Accuracy', 3: 'Mean_Val_Accuracy', 4: 'Mean_Test_Accuracy'}, inplace=True)

In [ ]:
Result_df.to_csv('Result_df.csv')

In [ ]:
#plt.figure(figsize=(20,12))
###plot of training set performance as a function of the number of folds used for training
fig, ax = plt.subplots()
for key, grp in Result_df.groupby(['Regularization Parameter Value']):
    ax = grp.plot(ax=ax, kind='line', x='Number of Folds', y='Mean_Train_Accuracy', label=key)
plt.legend(loc='best')
#plt.figure(figsize=(20,12))
plt.title('plot of training set performance as a function of the number of folds used for training')
plt.ylabel('Mean FVAF Value for training data')
plt.show()

In [ ]:
###Create a plot of test set performance as a function of the number of folds used for training

fig, ax = plt.subplots()
for key, grp in Result_df.groupby(['Regularization Parameter Value']):
    ax = grp.plot(ax=ax, kind='line', x='Number of Folds', y='Mean_Test_Accuracy', label=key)
plt.legend(loc='best')
#plt.figure(figsize=(20,12))
plt.title('plot of test set performance as a function of the number of folds used for training')
plt.ylabel('Mean FVAF Value for test data')
plt.show()

In [ ]:
max_review_length = 50
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [ ]:
# embedding_vecor_length = 32
# model = Sequential()
# model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
# model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.2))
# model.add(LSTM(100))
# model.add(Dropout(0.2))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model.summary())
# model.fit(X_train, y_train, epochs=5, batch_size=64)

embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
#model.add(Bidirectional(LSTM(100)))
# model.add(LSTM(100, recurrent_dropout = 0.2))
model.add(Bidirectional(LSTM(100, recurrent_dropout=0.2)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
# model.fit(X_train, y_train, epochs=5, batch_size=64)

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
## ROC curve
def generate_roc(model, ins, outs, ins_validation, outs_validation):
    '''
    Produce a ROC plot given a model, a set of inputs and the true outputs
    '''
    # Compute probabilistic predictions given images
    pred = model.predict(ins)
    # Compute false positive rate & true positive rate + AUC
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(outs, pred)
    auc = sklearn.metrics.auc(fpr, tpr)
    
    # Compute probabilistic predictions given images
    pred_val = model.predict(ins_validation)
    # Compute false positive rate & true positive rate + AUC
    fpr_val, tpr_val, thresholds_val = sklearn.metrics.roc_curve(outs_validation, pred_val)
    auc_val = sklearn.metrics.auc(fpr_val, tpr_val)
    
    
    # Generate the plot
    plt.figure(1)
    plt.axis('equal')
    plt.plot([0,1], [0,1], 'k--')
    plt.plot(fpr, tpr, 'b', label='Train AUC = {:.3f}'.format(auc))
    plt.plot(fpr_val, tpr_val, 'r', label='Test AUC = {:.3f}'.format(auc_val))
    plt.legend(loc='best')
    plt.title('ROC Curve for CNN + LSTM')
    plt.xlabel('FPR', fontsize=FONTSIZE)
    plt.ylabel('TPR', fontsize=FONTSIZE)